# AI stock analysis

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PauliusU/AI-stock-analysis/blob/master/AI-stock-analysis.ipynb)

❗Work in progress, some cells may not work❗


**SUMMARY, GOALS AND METHODOLOGY**

Project aims to use compare 3 different approaches to predict stock prices using AI and choose the best one. Two of them are based on neural networks one is a linear model.

All models are trained with the same data set - price of Tesla shares for the last 5 years:
1. First approach is price prediction using `ARIMA (Auto Regressive Integrated Moving Average)` model. ❗ N.B. ARIMA was not used during our AI course ❗
2. Another approach uses `LSTM (Long Short-Term memory) neural network` — the most popular machine learning approach for stock market prediction.
3. Final approach utilizes `GRU (Gated Recurrent Unit)` network.

Final comparison of each approach is done on the basis of `RMSE (root-mean-square error)`. The lower the error, the better the model is.

## Get data

This section uses Yahoo Finance API to fetch share price data for the last 5 years and visualizes result in the plot.

In [ ]:
# Use Yahoo Finance's API wrapper
!pip install yfinance
import yfinance as yf

print(f"yfinance version: {yf.__version__}")

In [ ]:
# Get stock data for the last 5 years

ticker = 'TSLA'
stock_data = yf.download(ticker, start='2017-11-01', end='2022-11-01')
print(stock_data.head())
print(stock_data.tail())

In [ ]:
# Plot closing price over time

import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.figure(figsize=(14, 7))
plt.title('Historical stock prices')
plt.plot(stock_data['Close'])
plt.xlabel('Time')
plt.ylabel('Closing price in USD')

## Prepare training and test sets

To prepare data to train and test models, only the closing price would be needed. Cell below gets 80% of these data points for closing price, normalizes them, and converts them into 2D array.

As we use LSTM neural network we also need to scale the data in the column ‘Close’ because the machine learning algorithm works much better with scaled than with regular data. Closing price is scaled in the range between 0 and 1. This is just the preliminary operation which we need to execute to let our model work with better efficiency.

In [ ]:
# Prepare training set

import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

close_prices = stock_data['Close']
values = close_prices.values

# training_data_len = math.ceil(len(values) * 0.8)
training_data_len = int(len(values) * 0.8)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(values.reshape(-1, 1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:
# Prepare test set

test_data = scaled_data[training_data_len - 60:, :]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

## Model#1 - ARIMA

Let's start by fitting a VERY simple ARIMA model to forecast the next value of the stock price.

In [ ]:
# Grid search ARIMA parameters for time series
from statsmodels.tsa.arima.model import ARIMA
import warnings
from math import sqrt
from sklearn.metrics import mean_squared_error


# Evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.8)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse, predictions, history


# Evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg, best_predictions, best_history, = float(
        "inf"), None, list(), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    rmse, predictions, history = evaluate_arima_model(dataset,
                                                                      order)
                    if rmse < best_score:
                        best_score, best_cfg, best_predictions, best_history = rmse, order, predictions, history
                    print(f"ARIMA{order} RMSE={rmse:.3f}")
                except:
                    continue
    print(f"Best ARIMA{best_cfg} RMSE={best_score:.3f}")
    return best_score, best_predictions, best_history


# Evaluate parameters
# p_values = [0, 1, 2, 4, 6, 8, 10]
# p_values = [0, 1, 2]
p_values = [0]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
rmse_arima, predictions_arima, history_arima = evaluate_models(values, p_values,
                                                               d_values,
                                                               q_values)

In [ ]:
import pprint

X = values
train_size = int(len(X) * 0.8)
train, test = X[0:train_size], X[train_size:]
history_arima = [x for x in train]
# pprint.pprint(train_size)
# pprint.pprint(train)
# pprint.pprint(test)
# pprint.pprint(history_arima)

# make predictions
predictions_arima = list()
for t in range(len(test)):
    print(f"{t}: {test[t]}")
    model = ARIMA(history_arima, order=(1, 0, 0))
    model_fit = model.fit()
    yhat = model_fit.forecast()[0]
    predictions_arima.append(yhat)
    history_arima.append(test[t])
# calculate out of sample error
rmse_arima = sqrt(mean_squared_error(test, predictions_arima))
print(rmse_arima)


## Model#2 - LSTM

### Set up LSTM model

In [ ]:
# Check if GPU is used

import tensorflow as tf

print(
    f"Number of GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

# Print the list of available training devices - alternative method to verify TensorFlow sees the GPU
print(tf.config.list_physical_devices())

In [ ]:
# Set up LSTM network architecture

from tensorflow import keras

model_lstm = keras.Sequential()
# 100 neurons on the first layer of LSTM
model_lstm.add(
    keras.layers.LSTM(100, return_sequences=True,
                      input_shape=(x_train.shape[1], 1)))
# 100 neurons on the second layer of LSTM
model_lstm.add(keras.layers.LSTM(100, return_sequences=False))
# Dense with 25 neurons
model_lstm.add(keras.layers.Dense(25))
# Just one neuron which is our result on the last step of the model
model_lstm.add(keras.layers.Dense(1))
model_lstm.summary()

# Set optimizer and loss function
# Use optimizer ‘adam’ which is the most popular in tasks of stock price prediction
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the LSTM model

# Teach model during 80 epochs (computations) with batch size on each epoch equal to 10 (1/10 of samples)
EPOCHS = 80
BATCH_SIZE = 10

# Show progress bar with verbose=1
history_lstm = model_lstm.fit(x_train, y_train, batch_size=BATCH_SIZE,
                              epochs=EPOCHS,
                              verbose=1)

In [ ]:
# Evaluation of LSTM

predictions_lstm = model_lstm.predict(x_test)
predictions_lstm = scaler.inverse_transform(predictions_lstm)
rmse_lstm = np.sqrt(np.mean(predictions_lstm - y_test) ** 2)
print(rmse_lstm)

## Model#3 - GRU

In [ ]:
# Set up GRU network architecture

model_gru = keras.Sequential()
model_gru.add(keras.layers.GRU(units=50, return_sequences=True,
                               input_shape=(x_train.shape[1], 1)))
model_gru.add(keras.layers.GRU(units=50))
model_gru.add(keras.layers.Dense(1))

# Use the same optimizer and loss function as with LSTM
# Use optimizer ‘adam’ which is the most popular in tasks of stock price prediction
model_gru.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the GRU model

# Teach model during 80 epochs (computations) with batch size on each epoch equal to 10 (1/10 of samples)
EPOCHS = 80
BATCH_SIZE = 10

# Use the same EPOCHS and BATCH_SIZE params as in LSTM model. Show progress bar with verbose=1,
history_gru = model_gru.fit(x_train, y_train, batch_size=BATCH_SIZE,
                            epochs=EPOCHS, verbose=1)

In [ ]:
# Evaluation of GRU model

predictions_gru = model_gru.predict(x_test)
predictions_gru = scaler.inverse_transform(predictions_gru)
rmse_gru = np.sqrt(np.mean(predictions_gru - y_test) ** 2)
print(rmse_gru)

## Conclusions

In [ ]:
import matplotlib as mpl

plt.style.use('ggplot')
plt.figure(figsize=(14, 7))

plt.plot(np.arange(len(history_lstm.history["loss"])) + 0.5,
         history_lstm.history["loss"], "b.-", label="Training loss (LSTM)")
plt.plot(np.arange(len(history_gru.history["loss"])) + 0.5,
         history_gru.history["loss"], "r.-", label="Training loss (GRU)")
plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
plt.title('Learning curves')
plt.legend(fontsize=14, loc='upper right')
plt.ylabel("Loss")
plt.grid(True)
plt.show()

In [ ]:
# Visualize prices the predicted by all models: ARIMA, LSTM and GRU

data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions_ARIMA'] = predictions_arima
validation['Predictions_LSTM'] = predictions_lstm
validation['Predictions_GRU'] = predictions_gru
plt.style.use('ggplot')
plt.figure(figsize=(14, 7))
plt.title('Predicted prices')
plt.xlabel('Date')
plt.ylabel('Closing price in USD')
plt.plot(train[-200:])  # Plot last 200 entries
plt.plot(validation[[
    'Close',
    'Predictions_ARIMA',
    'Predictions_LSTM',
    'Predictions_GRU'
]])
plt.legend([
    'Training prices',
    f'Actual prices for {ticker}',
    'Predicted prices (ARIMA)',
    'Predicted prices (LSTM)',
    'Predicted prices (GRU)'
],
    loc='upper left',
    fontsize=14)
plt.show()